## Import statements

In [1]:
import sys
sys.path.insert(0, '/Users/ahell/Documents/Python Projects/AIToolkit')

import os
from dotenv import load_dotenv, find_dotenv
import ai_toolkit as ai
from IPython.display import display, Markdown, HTML, clear_output

In [2]:
# Load in the environment variables
load_dotenv(find_dotenv())

True

In [3]:
ai.tools.AIModel.ALL_MODEL_NAMES

['gpt-3.5-turbo',
 'gpt-4',
 'gpt-3.5-turbo-16k',
 'claude-1',
 'claude-1-100k',
 'claude-instant-1',
 'claude-instant-1-100k',
 'claude-2',
 'respell-gpt-4-wrapper']

In [4]:
client_survey_id = 'adf4f406-757c-4e7d-b881-add3de4bbaf4'
client_name = 'Marcfirst'

## Dimension Analysis

In [5]:
request_url = f"https://dev-api.insite.ledgestone.com/api/results/comments/{client_survey_id}/nps_justification"
request_method = "GET"
request_headers = {
    "Authorization": f"Bearer {os.environ['BEARER_TOKEN']}",
    "Content-Type": "application/json"
}
comments_request = ai.io.APIRequest("Comments Request").set_input(
    url=request_url, method=request_method, headers=request_headers)
comments_request.process()

Comments Request (APIRequest)

In [6]:
comments_function = ai.operations.ExtractKey("Comments List Extractor").set_input(
    input=comments_request, key_name="comments")
comments_function.process()
comments_list = comments_function.get_output()
print(comments_list)

[{'prompt': 'What specific factors or experiences make you highly likely to recommend our company as a great place to work?', 'comment': 'Working with great residents ', 'nps_category': 'promoter', 'comment_number': 1, 'prompt_response_id': '00fe91ed-7aba-4605-94be-bbe8237a00e7'}, {'prompt': 'What specific factors or experiences make you highly likely to recommend our company as a great place to work?', 'comment': 'I love our values and mission. I believe leadership truly cares about me.', 'nps_category': 'promoter', 'comment_number': 2, 'prompt_response_id': '0858793c-79a5-4665-8d59-e6f3926ec6c8'}, {'prompt': 'What would our company need to do in order to get a rating of extremely likely?', 'comment': "Rather than spend money on rebranding the company, invest in creating a non-toxic work environment.  There is a sense of exclusion among non-leadership staff and morale continues to suffer as a result.  Bring in a neutral third party to conduct a comprehensive assessment of our workplac

In [7]:
comment_dict_input = ai.operations.Passthrough("Comment Dict Input").set_input(
    name_of_input="comment_dict")

class CreateMetadata(ai.CustomCodeBlock):
    def __init__(self, name):
        super().__init__(name)
        self.required_input = ["comment_dict"]

    def code(self):
        comment_dict = self.get_input("comment_dict")
        return {
            "client_survey_id": client_survey_id,
            "source": "QualitativeAnalyzer.ipynb",
            "function": "NPS Followup Analysis",
            "prompt_response_id": comment_dict["prompt_response_id"],
            "comment_number": str(comment_dict["comment_number"])
        }
    
create_metadata = CreateMetadata("Create Metadata").set_input(
    comment_dict=comment_dict_input)

class CreateReplacements(ai.CustomCodeBlock):
    def __init__(self, name):
        super().__init__(name)
        self.required_input = ["comment_dict"]

    def code(self):
        comment_dict = self.get_input("comment_dict")
        return {
            "comment": comment_dict["comment"]
        }
    
replacements_for_llm = CreateReplacements("Create Replacements").set_input(
    comment_dict=comment_dict_input)

dimension_analyzer_llm = ai.composed.LLMWithPromptRetrievalAndTracking("Dimension Analyzer LLM").set_input(
    template_name="Comment Dimension Analyzer",
    promptlayer_tags=[client_name],
    replacements=replacements_for_llm,
    model_name="gpt-4",
    metadata=create_metadata)

dimension_analyzer_json = ai.operations.ConvertToJSON("Dimension Analyzer JSON").set_input(
    input=dimension_analyzer_llm)

class ConvertSupportAndRecognitionToValue(ai.CustomCodeBlock):
    def __init__(self, name):
        super().__init__(name)
        self.required_input = ["analysis"]

    def code(self):
        analysis = self.get_input("analysis")
        if 'Support & Recognition' in analysis:
            analysis['Value'] = analysis.pop('Support & Recognition')
        return analysis

convert_support_and_regognition_to_value = ConvertSupportAndRecognitionToValue("Convert Support & Recognition to Value").set_input(
    analysis=dimension_analyzer_json)

class DimensionAnalyzerOutput(ai.CustomCodeBlock):
    def __init__(self, name):
        super().__init__(name)
        self.required_input = ["comment_dict", "analysis"]

    def code(self):
        comment_dict = self.get_input("comment_dict")
        analysis = self.get_input("analysis")
        output = list()
        for dimension, anal in analysis.items():
            output.append({
                "comment_number": comment_dict["comment_number"],
                "comment": comment_dict["comment"],
                "prompt_response_id": comment_dict["prompt_response_id"],
                "dimension": dimension,
                "analysis": anal["analysis"],
                "sentiment": anal["sentiment"]
                })

        return output

dimension_analyzer_output = DimensionAnalyzerOutput("Dimension Analyzer Output Function").set_input(
    comment_dict=comment_dict_input, analysis=convert_support_and_regognition_to_value)


dimension_analyzer = ai.AIProcess("Dimension Analyzer")
dimension_analyzer.expose_input("comment_dict", comment_dict_input)
dimension_analyzer.expose_output(dimension_analyzer_output)

Dimension Analyzer (AIProcess)

In [8]:
output = list()
for i in range(len(comments_list)):
    clear_output(wait=True)
    print(f"Processing comment {i+1} of {len(comments_list)}")
    dimension_analyzer.set_input(comment_dict=comments_list[i])
    dimension_analyzer.process()
    output.extend(dimension_analyzer.get_output())

Processing comment 1 of 66


AINonRetryableError: Cannot process AIProcess because not all inputs are present.
                                       Required inputs: []
Dynamic inputs: ['comment_dict']

In [10]:
dimension_analyzer.can_process()

In [ ]:
dimension_results_json = ai.io.FileWriter("Save Dimension Analysis JSON").set_input(file_path=f"Clients/{client_name}/dimensions_json.json", data=output)
dimension_results_json.process()
dimension_results_json.get_output()

'File successfully written'

## Driver Analysis

In [ ]:
dimensions_to_analyze = [
    "Accomplishment",
    "Alignment",
    "Compensation & Benefits",
    "Connection",
    "Diversity",
    "Purpose",
    "Security",
    "Value",
    "Work Environment",
    "Workload & Stress"
]
print(len(output))
dimension_ouput_for_driver_analysis = list()
for comment_analysis in output:
    dimension = comment_analysis["dimension"]
    if dimension in dimensions_to_analyze:
        dimension_ouput_for_driver_analysis.append(comment_analysis)
    else:
        print(f"Skipping {dimension}")

print(len(dimension_ouput_for_driver_analysis))

81
Skipping Not Applicable
Skipping Not Applicable
Skipping Not Applicable
78


In [ ]:
comment_dict_input = ai.operations.Passthrough("Comment Dict Input").set_input(
    name_of_input="comment_dict")

extract_comment = ai.operations.ExtractKey("Extract Comment").set_input(
    input=comment_dict_input, key_name="comment")

extract_comment_num = ai.operations.ExtractKey("Extract Comment Number").set_input(
    input=comment_dict_input, key_name="comment_number")

extract_prompt_response_id = ai.operations.ExtractKey("Extract Prompt Response ID").set_input(
    input=comment_dict_input, key_name="prompt_response_id")

extract_dimension = ai.operations.ExtractKey("Extract Dimension").set_input(
    input=comment_dict_input, key_name="dimension")

extract_analysis = ai.operations.ExtractKey("Extract Analysis").set_input(
    input=comment_dict_input, key_name="analysis")

extract_sentiment = ai.operations.ExtractKey("Extract Sentiment").set_input(
    input=comment_dict_input, key_name="sentiment")

driver_prompt_file_name_function = """
    dimension = get_input("dimension")
    return f"driver_prompt_{dimension.lower().replace(' ', '_').replace('&', 'and')}.txt"
"""

driver_prompt_file_name = ai.tools.Function("Driver Prompt File Name").set_input(
    function=driver_prompt_file_name_function, dimension=extract_dimension)

driver_prompt_file = ai.io.FileReader("Driver Analyzer File").set_input(
    file_path=driver_prompt_file_name)

driver_prompt = ai.tools.PromptBuilder("Driver Analyzer Prompt").set_input(
    template=driver_prompt_file, comment=extract_comment, value_analysis=extract_analysis)

driver_analyzer_llm = ai.tools.AIModel("Driver Analyzer LLM").set_input(
    model_name="gpt-4", prompt=driver_prompt)

driver_analyzer_json = ai.operations.ConvertToJSON("Driver Analyzer JSON").set_input(
    input=driver_analyzer_llm)

driver_analyzer_output_function = """
    comment_num = get_input("comment_number")
    comment = get_input("comment")
    prompt_response_id = get_input("prompt_response_id")
    dimension = get_input("dimension")
    analysis = get_input("analysis")
    sentiment = get_input("sentiment")
    takeaway = get_input("takeaway")
    output = list()
    for driver, takeaway in takeaway.items():
        output.append({
            'prompt_response_id': prompt_response_id,
            'comment_num': comment_num,
            'dimension': dimension,
            'sentiment': sentiment,
            'comment': comment,
            'analysis': analysis,
            'driver': driver,
            'takeaway': takeaway
        })

    return output
"""

driver_analyzer_output = ai.tools.Function("Driver Analyzer Output").set_input(
    function=driver_analyzer_output_function, 
    comment_number=extract_comment_num,
    comment=extract_comment,
    prompt_response_id=extract_prompt_response_id,
    dimension=extract_dimension,
    analysis=extract_analysis,
    sentiment=extract_sentiment,
    takeaway=driver_analyzer_json)

driver_analyzer = ai.AIProcess("Driver Analyzer")
driver_analyzer.expose_input("comment_dict", comment_dict_input)
driver_analyzer.expose_output(driver_analyzer_output)

Driver Analyzer (AIProcess)

In [ ]:
driver_output = list()
for i in range(len(dimension_ouput_for_driver_analysis)):
    clear_output(wait=True)
    print(f"Processing comment {i+1} of {len(dimension_ouput_for_driver_analysis)}")
    driver_analyzer.set_input(comment_dict=dimension_ouput_for_driver_analysis[i])
    driver_analyzer.process()
    driver_output.extend(driver_analyzer.get_output())

Processing comment 12 of 78
Processing Comment Dict Input (Passthrough)
Processing Extract Comment (ExtractKey)
Processing Extract Comment Number (ExtractKey)
Processing Extract Prompt Response ID (ExtractKey)
Processing Extract Dimension (ExtractKey)
Processing Driver Prompt File Name (Function)
Processing Driver Analyzer File (FileReader)
Processing Extract Analysis (ExtractKey)
Processing Extract Sentiment (ExtractKey)
Processing Driver Analyzer Prompt (PromptBuilder)
Processing Driver Analyzer LLM (AIModel)


RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-1ra0eeBjtT69TLEagEMEAA9n on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues.

In [ ]:
import json
display(driver_analyzer_llm.get_output())

data = '{"Pride": "From this employee\'s experience, the uncertainty and fear of job security is negatively affecting their pride in the organization; therefore, leaders need to address these concerns transparently and promptly because such uncertainties can divert focus from the core purpose of serving families."}'
json.loads(data)

'"Pride": "From this employee\'s experience, the uncertainty and fear of job security is negatively affecting their pride in the organization; therefore, leaders need to address these concerns transparently and promptly because such uncertainties can divert focus from the core purpose of serving families."'

{'Pride': "From this employee's experience, the uncertainty and fear of job security is negatively affecting their pride in the organization; therefore, leaders need to address these concerns transparently and promptly because such uncertainties can divert focus from the core purpose of serving families."}

In [ ]:
driver_results_json = ai.io.FileWriter("Save Driver Analysis JSON").set_input(file_path=f"Clients/{client_name}/drivers_json.json", data=driver_output)
driver_results_json.process()
driver_results_json.get_output()

In [ ]:
keep_keys = ['Meaning', 'Pride', 'Significance', 'Supervisor', 'Organization', 'Appreciation', 'Team', 'Empowerment', 'Investment', 'Impact', 'Clarity', 'Advancement', 'Strategy', 'Vision', 'Mastery', 'Trust', 'Equity', 'Execution']
extra_keep_keys = ['Compensation', 'Work Environment', 'Workload & Stress', 'Diversity']

result_dict = {}
for anal in driver_output:
        if not (anal['driver'] in keep_keys or anal['driver'] in extra_keep_keys):
            continue
        dimension = row['Comment']['dimension']
        comment = row['Comment']['comment']
        analysis = row['Comment']['analysis']
        comment_num = row['Comment']['comment_number']
        takeaway = row['Takeaway']
        for key, value in takeaway.items():
            if key in result_dict:
                result_dict[key].append(f'{comment_num} - {value}')
            else:
                result_dict[key] = [f'{comment_num} - {value}']